In [1]:
#crawl_iwencai_flags_history
from opentrader.core.crawler import *
from opentrader.agents.xueqiu.api import *
from opentrader.agents.ths.api import *
from opentrader.core.ticker import *
from opentrader.common.db import db_ot
from opentrader.common.utils import d2dt
from pymongo.errors import *

ths = THSAPI()
start_date = gen_time('2015-01-01 00:00:00').date()
end_date = gen_time('2015-12-16 00:00:00').date()
date = start_date
dates = []
while date <= end_date:
    if TradeCalendar.check_date(date):
        dates.append(date)
    date += timedelta(1)

In [2]:
# now use dates to fetch history flags data and store into mongodb
for date in dates:
    f = open('iwencai_flags.json', 'r')
    try:
        flags = json.loads(f.read())
    except:
        flags = []
    i = 0
    for flag in flags:
        # search each flag at iwencai
        result = ths.query_iwencai(date.strftime('%Y年%m月%d日')+flag)
        result = result['result']
        # insert into db
        i += 1
        print(i)
        try:
            db_ot.iwencai_flag_history.insert_one({'flag':flag, 'date':d2dt(date),'symbols':[each[0] for each in result]})
        except DuplicateKeyError as e:
            print(e)

http://www.iwencai.com/stockpick/search?typed=1&preParams=&ts=1&perpage=4000&f=1&qs=1&selfsectsn=&querytype=&searchfilter=&tid=stockpick&w=2015年01月05日周线kdj超卖
1
E11000 duplicate key error index: opentrader.iwencai_flag_history.$flag_1_date_1 dup key: { : "周线kdj超卖", : new Date(1420416000000) }
http://www.iwencai.com/stockpick/search?typed=1&preParams=&ts=1&perpage=4000&f=1&qs=1&selfsectsn=&querytype=&searchfilter=&tid=stockpick&w=2015年01月05日月线kdj超卖
2
E11000 duplicate key error index: opentrader.iwencai_flag_history.$flag_1_date_1 dup key: { : "月线kdj超卖", : new Date(1420416000000) }
http://www.iwencai.com/stockpick/search?typed=1&preParams=&ts=1&perpage=4000&f=1&qs=1&selfsectsn=&querytype=&searchfilter=&tid=stockpick&w=2015年01月05日skdj超卖
3
E11000 duplicate key error index: opentrader.iwencai_flag_history.$flag_1_date_1 dup key: { : "skdj超卖", : new Date(1420416000000) }
http://www.iwencai.com/stockpick/search?typed=1&preParams=&ts=1&perpage=4000&f=1&qs=1&selfsectsn=&querytype=&searchfilter=&

KeyboardInterrupt: 